# Benchmark QRT

This notebook illustrates a simple benchmark example that should help novice participants to start the competition.

## Used libraries

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

## Loading data

The train and test inputs are composed of 46 features.

The target of this challenge is `RET` and corresponds to the fact that the **return is in the top 50% of highest stock returns**.

Since the median is very close to 0, this information should not change much with the idea to predict the sign of the return.

In [2]:
x_train = pd.read_csv('../data/x_train.csv', index_col='ID')
y_train = pd.read_csv('../data/y_train.csv', index_col='ID')
train = pd.concat([x_train, y_train], axis=1)
test = pd.read_csv('../data/x_test.csv', index_col='ID')
train.head()

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET
ID,,,,,,,,,,,,,,,,,,,,,
0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,0.179183,...,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937,True
1,0,3,43,15,6,104,0.003984,NaN,-0.090580,NaN,...,NaN,0.003774,NaN,-0.018518,NaN,-0.028777,NaN,-0.034722,NaN,True
2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,0.084771,...,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868,False
3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,-0.089919,...,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458,False
4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,-0.943033,...,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806,False


## Feature Engineering

The main drawback in this challenge would be to deal with the noise. To do that, we could create some feature that aggregate features with some statistics. 

The following cell computes statistics on a given target conditionally to some features. For example, we want to generate a feature that describe the mean of `RET_1` conditionally to the `SECTOR` and the `DATE`.

**Ideas of improvement**: change shifts, the conditional features, the statistics, and the target. 

In [3]:
# Feature engineering
new_features = []

# Conditional aggregated features
shifts = [i for i in range(1,21)]  # Choose some different shifts
statistics = ['mean']  # the type of stat

# GROUP BY 4 features only for "RET", shifts = [1,...,21]
gb_features = ['STOCK','SECTOR','INDUSTRY', 'DATE']
target_feature = ['RET']
tmp_name = '_'.join(gb_features)
for j in target_feature:
    for shift in shifts:
        for stat in statistics:
            name = f'{j}_{shift}_{tmp_name}_{stat}'
            feat = f'{j}_{shift}'
            new_features.append(name)
            for data in [train, test]:
                data[name] = data.groupby(gb_features)[feat].transform(stat)
           
        
# # GROUP BY "DATE" for features['RET','VOLUME'], shifts=[1,...,21]
# gb_features = ['DATE']
# target_features = ['RET','VOLUME']
# tmp_name = '_'.join(gb_features)
# for j in target_features:
# #     target_feature = j
#     for shift in shifts:
#         for stat in statistics:
#             name = f'{j}_{shift}_{tmp_name}_{stat}'
#             feat = f'{j}_{shift}'
#             new_features.append(name)
#             for data in [train, test]:
#                 data[name] = data.groupby(gb_features)[feat].transform(stat)

        
# # GROUP BY "STOCK" for features['RET','VOLUME'], shifts=[1,...,21]
# gb_features = ['STOCK']
# target_features = ['RET','VOLUME']
# tmp_name = '_'.join(gb_features)
# for j in target_features:
# #     target_feature = j
#     for shift in shifts:
#         for stat in statistics:
#             name = f'{j}_{shift}_{tmp_name}_{stat}'
#             feat = f'{j}_{shift}'
#             new_features.append(name)
#             for data in [train, test]:
#                 data[name] = data.groupby(gb_features)[feat].transform(stat)

                
gb_features = ['STOCK','SECTOR','INDUSTRY', 'DATE']
target_feature = "VOLUME"
shifts = [1,13]
tmp_name = '_'.join(gb_features)
for shift in shifts:
    for stat in statistics:
        name = f'{target_feature}_{shift}_{tmp_name}_{stat}'
        feat = f'{target_feature}_{shift}'
        new_features.append(name)
        for data in [train, test]:
            data[name] = data.groupby(gb_features)[feat].transform(stat)
            


## Feature selection

To reduce the number of feature (and the noise) we only consider the 5 last days of `RET` and `VOLUME` in addition to the newly created feature.

In [4]:
target = 'RET'

n_shifts = [1,2,3,7,14,17]
features = ['RET_%d' % i for i in n_shifts]
# features += ['VOLUME_1','DATE']
features += ['VOLUME_1']

features += new_features  # The conditional features
train[features].head()

,RET_1,RET_2,RET_3,RET_7,RET_14,RET_17,VOLUME_1,RET_1_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_2_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_3_STOCK_SECTOR_INDUSTRY_DATE_mean,...,RET_13_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_14_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_15_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_16_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_17_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_18_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_19_STOCK_SECTOR_INDUSTRY_DATE_mean,RET_20_STOCK_SECTOR_INDUSTRY_DATE_mean,VOLUME_1_STOCK_SECTOR_INDUSTRY_DATE_mean,VOLUME_13_STOCK_SECTOR_INDUSTRY_DATE_mean
ID,,,,,,,,,,,,,,,,,,,,,
0,-0.015748,-0.015504,0.010972,-0.017215,-0.049370,0.003254,0.147931,-0.015748,-0.015504,0.010972,...,0.027624,-0.049370,-0.028571,0.059459,0.003254,0.008752,-0.012959,-0.002155,0.147931,-0.244636
1,0.003984,-0.090580,0.018826,-0.026756,-0.052044,0.003774,NaN,0.003984,-0.090580,0.018826,...,-0.019608,-0.052044,-0.004073,0.015413,0.003774,-0.018518,-0.028777,-0.034722,NaN,NaN
2,0.000440,-0.058896,-0.009042,-0.023047,-0.002686,-0.017612,-0.096282,0.000440,-0.058896,-0.009042,...,-0.009426,-0.002686,-0.007552,0.008964,-0.017612,-0.006562,-0.012101,-0.006867,-0.096282,-0.081783
3,0.031298,0.007756,-0.004632,-0.043962,0.000479,0.033824,-0.429540,0.031298,0.007756,-0.004632,...,-0.010067,0.000479,0.021057,-0.031769,0.033824,-0.001468,-0.013520,-0.036745,-0.429540,-0.839232
4,0.027273,-0.039302,0.000000,-0.026549,0.095891,-0.012659,-0.847155,0.027273,-0.039302,0.000000,...,-0.041667,0.095891,-0.026667,-0.038461,-0.012659,0.004237,0.004256,-0.040817,-0.847155,0.271702


## Model and local score

A Random Forest (RF) model is chosen for the Benchmark. We consider a large number of tree with a quiet small depth. The missing values are simply filled with 0. A KFold is done on the dates (using `DATE`) for a local scoring of the model. 

**Ideas of improvements**: Tune the RF hyperparameters, deal with the missing values, change the features, consider another model, ...

In [5]:
X_train = train[features]
y_train = train[target]

# A quiet large number of trees with low depth to prevent overfits
rf_params = {
    'n_estimators': 500,
    'max_depth': 2**3,
    'random_state': 0,
    'n_jobs': -1
}

train_dates = train['DATE'].unique()
test_dates = test['DATE'].unique()

n_splits = 4
scores = []

models = []

splits = KFold(n_splits=n_splits, random_state=0,
               shuffle=True).split(train_dates)

for i, (local_train_dates_ids, local_test_dates_ids) in enumerate(splits):
    local_train_dates = train_dates[local_train_dates_ids]
    local_test_dates = train_dates[local_test_dates_ids]

    local_train_ids = train['DATE'].isin(local_train_dates)
    local_test_ids = train['DATE'].isin(local_test_dates)

    X_local_train = X_train.loc[local_train_ids]
    y_local_train = y_train.loc[local_train_ids]
    X_local_test = X_train.loc[local_test_ids]
    y_local_test = y_train.loc[local_test_ids]

    X_local_train = X_local_train.fillna(0)
    X_local_test = X_local_test.fillna(0)

#     model = RandomForestClassifier(**rf_params)
    model = LogisticRegression()
    model.fit(X_local_train, y_local_train)

    y_local_pred = model.predict_proba(X_local_test)[:, 1]
    
    sub = train.loc[local_test_ids].copy()
    sub['pred'] = y_local_pred
    y_local_pred = sub.groupby('DATE')['pred'].transform(lambda x: x > x.median()).values

    models.append(model)
    score = accuracy_score(y_local_test, y_local_pred)
    scores.append(score)
    print(f"Fold {i+1} - Accuracy: {score* 100:.2f}%")

mean = np.mean(scores)*100
std = np.std(scores)*100
u = (mean + std)
l = (mean - std)
print(f'Accuracy: {mean:.2f}% [{l:.2f} ; {u:.2f}] (+- {std:.2f})')

Fold 1 - Accuracy: 50.53%
Fold 2 - Accuracy: 51.30%
Fold 3 - Accuracy: 50.84%
Fold 4 - Accuracy: 50.23%
Accuracy: 50.73% [50.33 ; 51.12] (+- 0.40)


## Generate the submission

The same parameters of the RF model are considered. With that we build a new RF model on the entire `train` dataset. The predictions are saved in a `.csv` file.

In [6]:
X_test = test[features]

# rf_params['random_state'] = 0
# model = RandomForestClassifier(**rf_params)
# model = 
# model.fit(X_train.fillna(0), y_train)
# y_pred = model.predict_proba(X_test.fillna(0))[:, 1]
y_pred = model.predict(X_test.fillna(0))[:, 1]

sub = test.copy()
sub['pred'] = y_pred
y_pred = sub.groupby('DATE')['pred'].transform(
    lambda x: x > x.median()).values

submission = pd.Series(y_pred)
submission.index = test.index
submission.name = target

submission.to_csv('./logistic.csv', index=True, header=True)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed


The local accuracy is around 51. If we did not overfit, we shall expect something within the range above.

After submitting the benchmark file at https://challengedata.ens.fr, we obtain a public score of 51.31 %.